Class Labels:
Traffic Light Signal
Stop Signal
Speedlimit Signal
Crosswalk Signal
Crosswalk
Pedestrian
Bus
Car
Truck


https://www.kaggle.com/datasets/princekhunt19/road-detection-imgs-and-labels

In [ ]:
# Install Ultralytics library
!pip install ultralytics

In [ ]:
import os
import numpy as np
import pandas as pd
import wandb
import yaml
import zipfile
import matplotlib.pyplot as plt
import cv2
from IPython.display import HTML
from matplotlib import animation
from tqdm import tqdm
from PIL import Image
from ultralytics import YOLO
from IPython.display import FileLink

In [ ]:
data_yaml = dict(
    train ='/kaggle/working/dataset/train/images',
    val ='/kaggle/working/dataset/val/images',
    test='/kaggle/working/dataset/test/images',
    nc = 9,
    names =['Trafic Light Signal', 'Stop Signal', 'Speedlimit Signal', 'Crosswalk Signal', 'Crosswalk', 'Pedestrian', 'Bus', 'Car', 'Truck']
)
with open('dataset.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [ ]:
def create_zip(source_folder, destination_zip):
    with zipfile.ZipFile(destination_zip, 'w', zipfile.ZIP_DEFLATED) as zip_ref:
        for root, dirs, files in os.walk(source_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_ref.write(file_path, arcname=os.path.relpath(file_path, source_folder))
def extract_zip(zip_file, destination_folder):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)
source_folder = '/kaggle/input/road-detection-imgs-and-labels'
destination_zip = '/kaggle/working/dataset.zip'
destination_folder = '/kaggle/working/'

create_zip(source_folder, destination_zip)
extract_zip(destination_zip, destination_folder)

In [ ]:
def create_zip(source_folder, destination_zip):
    with zipfile.ZipFile(destination_zip, 'w', zipfile.ZIP_DEFLATED) as zip_ref:
        for root, dirs, files in os.walk(source_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_ref.write(file_path, arcname=os.path.relpath(file_path, source_folder))
def extract_zip(zip_file, destination_folder):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)
source_folder = '/kaggle/input/road-detection-imgs-and-labels'
destination_zip = '/kaggle/working/dataset.zip'
destination_folder = '/kaggle/working/'

create_zip(source_folder, destination_zip)
extract_zip(destination_zip, destination_folder)

In [ ]:
model = YOLO("yolo11m.pt") 

In [ ]:
!yolo task=detect mode=train model=yolo11m.pt data=dataset.yaml epochs=25 imgsz=640

In [ ]:
!zip -r runs.zip runs/

In [ ]:
def display_training_results(directory, file_extension=('.jpg', '.png'), images_per_row=2, image_height=10):
    image_paths = []
    for dirname, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith(file_extension):
                image_paths.append(os.path.join(dirname, filename))
    image_paths = sorted(image_paths)

    num_images = len(image_paths)
    num_rows = (num_images + images_per_row - 1) // images_per_row  
    figsize = (images_per_row * image_height, num_rows * image_height)
    
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=figsize)
    axes = axes.flatten() 

    for i, path in enumerate(image_paths):
        image = Image.open(path)
        axes[i].imshow(np.array(image))
        axes[i].axis('off') 

    for j in range(len(image_paths), len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()
display_training_results('/kaggle/working/runs/detect/train')

In [ ]:
best_path = 'runs/detect/train/weights/best.pt'
source = 'dataset/test/images'

image_paths = []
for dirname, _, filenames in os.walk(source):
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('.png'): 
            image_paths.append(os.path.join(dirname, filename))
image_paths = sorted(image_paths)

model = YOLO(best_path)
!yolo task=detect mode=predict model={best_path} conf=0.1 source={source}

In [ ]:
results = model.predict(source, conf=0.1)
results

In [ ]:
paths = []

for dirname, _, filenames in os.walk(source):
    for filename in filenames:
        if filename[-4:] == '.jpg' or filename[-4:] == '.png':
            paths += [(os.path.join(dirname, filename))]
            
paths = sorted(paths)
df = pd.DataFrame(columns = range(6))

for i in range(len(results)):
    arri = pd.DataFrame(results[i].boxes.data.cpu().numpy()).astype(float)
    path = paths[i]
    file = path.split('/')[-1]
    arri = arri.assign(file = file)
    arri = arri.assign(i = i)
    df = pd.concat([df, arri], axis = 0)
    
df.columns=['x','y','x2','y2','confidence','class','file','i']
display(df)

In [ ]:
def draw_box(image_index):
    image_path = paths[image_index]
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    filename = image_path.split('/')[-1]
    
    if not df[df['file'] == filename].empty:
        boxes = df[df['file'] == filename].reset_index(drop=True)

        for box_index in range(len(boxes)):
            label = boxes.loc[box_index, 'class']
            x = int(boxes.loc[box_index, 'x'])
            y = int(boxes.loc[box_index, 'y'])
            x2 = int(boxes.loc[box_index, 'x2'])
            y2 = int(boxes.loc[box_index, 'y2'])
            
            cv2.putText(
                image, f'{label}', (x, int(y - 4)),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2
            )
            cv2.rectangle(image, (x, y), (x2, y2), (0, 255, 0), 2) 
    
    return image
def create_animation(images):
    fig = plt.figure(figsize=(12, 8))
    image_display = plt.imshow(cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB))
    text = plt.text(0.05, 0.05, f'Slide {0}', transform=fig.transFigure, fontsize=14, color='blue')
    plt.axis('off')
    plt.close()

    def animate_func(frame_index):
        image_display.set_array(cv2.cvtColor(images[frame_index], cv2.COLOR_BGR2RGB))
        text.set_text(f'Slide {frame_index}')
        return [image_display]

    return animation.FuncAnimation(fig, animate_func, frames=len(images), interval=1000)
annotated_images = []

for index in tqdm(range(len(paths))):
    annotated_images.append(draw_box(index))

In [ ]:
anim = create_animation(annotated_images)
HTML(anim.to_jshtml())